# Confusion matrix for CocoAPI

The "main_Evaluate_Object_Detection.ipynb" uses the CocoApi to evaluate the model performance, e.g., AP50. But that code does not include the output of confusion matrix (e.g., TP, FP, and FN).

This script provides the output of confusion matrix using **FiftyOne** tool

FiftyOne’s implementation of COCO-style evaluation matches the reference implementation available via pycocotools (https://github.com/cocodataset/cocoapi).

Refer to: 
https://docs.voxel51.com/integrations/coco.html#loading-coco-formatted-data

## Install fiftyone 

In [1]:
#   For windows:
#        pip3 install fiftyone --user

## Load coco dataset (ground-truth)

**Note**: 

(1) the folder/path of images in json file must contains " / " instead of " \ " 

(2) In json file, "image_id" and the "id" for annotations must not start "0", otherwise the results will be not correct

In [2]:
import fiftyone as fo

# A name for the dataset
# name = "my-dataset"

# GJO test dataset
image_path = r"U:\AIMMW\Noria Upload\Data Tianlong Borgharen\Dataset_labels\Location_5410\0110\tiles_224\images_with_litter"
labels_path = r"U:\AIMMW\Noria Upload\Data Tianlong Borgharen\Dataset_labels\Location_5410\0110\tiles_224\labels_coco\test.json"

# Amsterdam test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Amsterdam\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Amsterdam\labels_coco\test.json"

# Groningen test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Groningen\test\tiles_224\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Groningen\test\tiles_224\labels_coco\SL_test_525.json"

# Vietnam test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Vietnam_3_class_V5\test\tiles_224\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Vietnam_3_class_V5\test\tiles_224\labels_coco\SL_test_1091.json"


# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
    # include_id=True,
    # name=name,
)
print(dataset)

 100% |█████████████████| 188/188 [1.3s elapsed, 0s remaining, 143.2 samples/s]         
Name:        2024.05.02.10.27.18
Media type:  image
Num samples: 188
Persistent:  False
Tags:        []
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [3]:
# show the first data in the dataset,
# Carefully check the "label" name!
# Note the coordinates of the bbox is scaled into [0,1]

print(dataset.first().detections.detections[0])

<Detection: {
    'id': '66334e66fcc289b3399cb5f0',
    'attributes': {},
    'tags': [],
    'label': 'litter',
    'bounding_box': [
        0.49107142857142855,
        0.3169642857142857,
        0.29464285714285715,
        0.48660714285714285,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'iscrowd': 0.0,
    'segmented': None,
    'pose': None,
    'truncated': None,
    'difficult': None,
}>


## Load predictions

When inferencing a model on a test dataset using Detectron2, it will output a prediction file named "coco_instances_results.json" with predicted bbox

Note: when load predictions in a json file, the "id" in "images" field must start from **"1"**, Otherwise it will not work. You may use to change the "id in json using: *https://github.com/TianlongJia/Tools/tree/master/Vision/Manage_pic_and_folder//Modify_JSON.ipynb*

In [4]:
import fiftyone.utils.coco as fouc
import json

#
# Mock COCO predictions, where:
# - `image_id` corresponds to the `coco_id` field of `coco_dataset`
# - `category_id` corresponds to classes in `coco_dataset.default_classes`
#
# predictions = [
#     {"image_id": 1, "category_id": 18, "bbox": [258, 41, 348, 243], "score": 0.87},
#     {"image_id": 2, "category_id": 11, "bbox": [61, 22, 504, 609], "score": 0.95},
# ]

json_file= r"U:\AIMMW\Noria Upload\Data Tianlong Borgharen\Dataset_labels\Location_5410\0110\tiles_224\test_results_score_0.8\test_results\coco_instances_results.json"

with open(json_file, 'r') as fcc_file:
    fcc_data = json.load(fcc_file)
# print(fcc_data)
    
predictions = fcc_data


# Add COCO predictions to `predictions` field of dataset
classes = dataset.default_classes
print("classes:", classes)
# classes = ['hold_space', 'litter',]
# classes = ['airplane', 'apple', ...]
fouc.add_coco_labels(dataset, "predictions", predictions, classes)

# Verify that predictions were added to two images
print(dataset.count("predictions"))  # 2

classes: ['0', 'litter']
94


In [5]:
predictions

[{'image_id': 1,
  'category_id': 1,
  'bbox': [113.94168090820312,
   69.7156753540039,
   58.04722595214844,
   107.30437469482422],
  'score': 0.9741113781929016},
 {'image_id': 4,
  'category_id': 1,
  'bbox': [96.66016387939453,
   166.22056579589844,
   38.565834045410156,
   57.77943420410156],
  'score': 0.9555864334106445},
 {'image_id': 6,
  'category_id': 1,
  'bbox': [103.65355682373047,
   86.33534240722656,
   42.62610626220703,
   90.02708435058594],
  'score': 0.8163942694664001},
 {'image_id': 12,
  'category_id': 1,
  'bbox': [51.70920944213867, 0.0, 35.98482131958008, 42.4447135925293],
  'score': 0.9311988353729248},
 {'image_id': 19,
  'category_id': 1,
  'bbox': [134.62960815429688,
   141.76358032226562,
   28.362640380859375,
   58.98420715332031],
  'score': 0.985340416431427},
 {'image_id': 24,
  'category_id': 1,
  'bbox': [100.31562805175781,
   57.72908401489258,
   39.231536865234375,
   58.32546615600586],
  'score': 0.9479544162750244},
 {'image_id': 26,

In [6]:
print(dataset.count("detections"))  # the number of images

188


In [7]:
# show the first data (with prediction bbox) in the dataset ,
# Carefully check the "label" name!
# Note the coordinates of the predicted bbox is scaled into [0,1]

# print(dataset.first())

In [8]:
# sample = dataset.first()
# print(sample.detections.detections[0])

## Output results 

by comparing ground-truth labels and predictions bbox

**Note**: Predicted and ground truth objects are matched using a specified IoU threshold (default = 0.50). This threshold can be customized via the iou parameter. The IoU selection affects the calculation of TP, FN, are FP value.

In [9]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Evaluate the objects in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    iou=0.5,
    method="coco",   
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |█████████████████| 188/188 [2.3s elapsed, 0s remaining, 77.8 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 188/188 [2.4s elapsed, 0s remaining, 92.5 samples/s]       


In [10]:
results

### mAP

Note: this is the mAP at IoU=.50:.05:.95

In [11]:
print(results.mAP())

0.11719223806748028


### TP, FP, FN

Note: the calculation of these metrics follows the IoU pre-defined above:

i.e., results = dataset.evaluate_detections(..., iou=XXX, ...)

In [12]:
# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

TP: 81
FP: 16
FN: 110


### Precision, recall, F1-score

Note: the calculation of these metrics follows the IoU pre-defined above:

i.e., results = dataset.evaluate_detections(..., iou=XXX, ...)

In [ ]:
classes = 1
counts = dataset.count_values("detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:classes]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

### Precision-recall curve

In [ ]:
plot = results.plot_pr_curves(classes=["litter"])
plot.show()

### Confusion matrix

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz


# Perform evaluation, allowing objects to be matched between classes
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    classwise=False,
)

# Generate a confusion matrix for the specified classes
plot = results.plot_confusion_matrix(classes=["litter"])
plot.show()